In [ ]:
!pip -q install pyspark==3.5.1 openpyxl reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.2 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [ ]:
from pyspark.sql import SparkSession
import pandas as pd, os, matplotlib.pyplot as plt
spark = SparkSession.builder.appName("PlayStoreAnalysis").getOrCreate()

In [ ]:
# Upload your Excel
from google.colab import files
up = files.upload('googlestore.xlsx')  # choose googlestore.xlsx
xlsx_name = list(up.keys())[0]



# Convert Excel → CSV (Spark reads CSV more robustly)
pdf = pd.read_excel(xlsx_name, sheet_name=0)
csv_path = "/content/googlestore.csv"
pdf.to_csv(csv_path, index=False)
print("Saved CSV at:", csv_path, " | rows:", len(pdf))

Saving googlestore.xlsx to googlestore.xlsx/googlestore.xlsx
Saving googleplaystore.csv to googlestore.xlsx/googleplaystore.csv
Saved CSV at: /content/googlestore.csv  | rows: 10841


In [ ]:
from pyspark.sql.types import *



schema = StructType([

    StructField("App", StringType(), True),

    StructField("Category", StringType(), True),

    StructField("Rating", DoubleType(), True),

    StructField("Reviews", LongType(), True),

    StructField("Size", StringType(), True),

    StructField("Installs", StringType(), True),

    StructField("Type", StringType(), True),

    StructField("Price", StringType(), True),

    StructField("Content Rating", StringType(), True),

    StructField("Genres", StringType(), True),

    StructField("Last Updated", StringType(), True),

    StructField("Current Ver", StringType(), True),

    StructField("Android Ver", StringType(), True),

])

df_raw = spark.read.option("header", True).schema(schema).csv(csv_path)
df_raw.printSchema()
df_raw.show(5, truncate=False)

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: long (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+----------------------------------------------------+--------------+------+-------+----+-----------+----+-----+--------------+-------------------------+-------------------+------------------+------------+
|App                                                 |Category      |Rating|Reviews|Size|Installs   |Type|Price|Content Rating|Genres                   |Last Updated       |Current Ver       |Android Ver |
+----------------------------------------------------+--------------+

In [ ]:
from pyspark.sql import functions as F
df = (
df_raw
.withColumn("installs_num", F.regexp_replace(F.col("Installs"), r"[+,]",
"").cast("long"))
.withColumn("price_num", F.regexp_replace(F.col("Price"), r"^\$",
"").cast("double"))
.withColumn("reviews_num", F.col("Reviews").cast("long"))
.withColumn("rating_num", F.col("Rating").cast("double"))
.withColumn("type_norm", F.initcap(F.trim(F.col("Type"))))
.withColumn("category_norm",F.trim(F.col("Category")))
.filter(F.col("App").isNotNull() & F.col("installs_num").isNotNull())
.filter((F.col("rating_num").isNull()) | ((F.col("rating_num") >= 0) &
(F.col("rating_num") <= 5)))
)
df.cache()
print("Rows after cleaning:", df.count())

Rows after cleaning: 10840


In [ ]:
from pyspark.sql import functions as F

# 1) Top 10 apps by reviews
top10_reviews = (
    df.orderBy(F.desc("reviews_num"))
      .limit(10)
)

# 2) Top 10 apps by installs
top10_installs = (
    df.orderBy(F.desc("installs_num"))
      .limit(10)
)

# 2b) Free vs Paid
type_dist_count = (
    df.groupBy("type_norm")
      .agg(F.count("*").alias("app_count"))
)

type_dist_installs = (
    df.groupBy("type_norm")
      .agg(F.sum(F.col("installs_num").cast("double")).alias("total_installs"))
)

# 3) Category-wise installs
category_installs = (
    df.groupBy("category_norm")
      .agg(
          F.count("*").alias("num_apps"),
          F.sum(F.col("installs_num").cast("double")).alias("total_installs"),
          F.avg(F.col("rating_num").cast("double")).alias("avg_rating")
      )
      .orderBy(F.desc("total_installs"))
)

# 4) Top paid by installs
paid_df = df.filter(F.col("type_norm") == "Paid")

top_paid_by_installs = (
    paid_df.orderBy(F.desc("installs_num"))
           .limit(10)
)

# 5) Top paid by rating (with review threshold)
MIN_REVIEWS = 1000

top_paid_by_rating = (
    paid_df.filter(F.col("reviews_num") >= MIN_REVIEWS)
           .orderBy(F.desc("rating_num"), F.desc("reviews_num"))
           .limit(10)
)

# Quick peek in notebook
for title, sdf in [
    ("Top 10 by Reviews", top10_reviews),
    ("Top 10 by Installs", top10_installs),
    ("Free vs Paid (Count)", type_dist_count),
    ("Free vs Paid (Installs)", type_dist_installs),
    ("Top Categories by Installs", category_installs),
    ("Top Paid by Installs", top_paid_by_installs),
    ("Top Paid by Rating", top_paid_by_rating),
]:
    print("====", title, "====")
    sdf.show(truncate=False)


==== Top 10 by Reviews ====
+------------------------------------------+-------------+------+--------+------------------+--------------+----+-----+--------------+-------------+-------------------+------------------+------------------+------------+---------+-----------+----------+---------+-------------+
|App                                       |Category     |Rating|Reviews |Size              |Installs      |Type|Price|Content Rating|Genres       |Last Updated       |Current Ver       |Android Ver       |installs_num|price_num|reviews_num|rating_num|type_norm|category_norm|
+------------------------------------------+-------------+------+--------+------------------+--------------+----+-----+--------------+-------------+-------------------+------------------+------------------+------------+---------+-----------+----------+---------+-------------+
|Facebook                                  |SOCIAL       |4.1   |78158306|Varies with device|1,000,000,000+|Free|0    |Teen          |Social 

In [ ]:
fig_dir = "/content/figs"
os.makedirs(fig_dir, exist_ok=True)

def barh_plot(sdf, x, y, title, fname):
    _pdf = sdf.toPandas()
    _pdf[y] = _pdf[y].fillna("Unknown").astype(str)   # fixed "Unknown&quot" typo
    plt.figure(figsize=(10,6))
    plt.barh(_pdf[y], _pdf[x])
    plt.gca().invert_yaxis()
    plt.title(title)
    plt.tight_layout()
    out = f"{fig_dir}/{fname}"
    plt.savefig(out, dpi=150)
    plt.close()
    return out

charts = {
    "Top 10 Apps by Reviews": barh_plot(
        top10_reviews, "reviews_num", "App",
        "Top 10 Apps by Reviews", "reviews.png"
    ),
    "Top 10 Apps by Installs": barh_plot(
        top10_installs, "installs_num", "App",
        "Top 10 Apps by Installs", "installs.png"
    ),
    "Free vs Paid (Count)": barh_plot(
        type_dist_count, "app_count", "type_norm",
        "Free vs Paid (Count)", "typecount.png"
    ),
    "Free vs Paid (Installs)": barh_plot(
        type_dist_installs, "total_installs", "type_norm",
        "Free vs Paid (Installs)", "typeinstalls.png"
    ),
    "Top Categories by Installs": barh_plot(
        category_installs.limit(10), "total_installs", "category_norm",
        "Top Categories by Installs", "categories.png"
    ),
    "Top Paid Apps by Installs": barh_plot(
        top_paid_by_installs, "installs_num", "App",
        "Top Paid Apps by Installs", "paid_installs.png"
    ),
    "Top Paid Apps by Rating": barh_plot(
        top_paid_by_rating, "rating_num", "App",
        "Top Paid Apps by Rating", "paid_rating.png"
    ),
}

charts


{'Top 10 Apps by Reviews': '/content/figs/reviews.png',
 'Top 10 Apps by Installs': '/content/figs/installs.png',
 'Free vs Paid (Count)': '/content/figs/typecount.png',
 'Free vs Paid (Installs)': '/content/figs/typeinstalls.png',
 'Top Categories by Installs': '/content/figs/categories.png',
 'Top Paid Apps by Installs': '/content/figs/paid_installs.png',
 'Top Paid Apps by Rating': '/content/figs/paid_rating.png'}

In [ ]:
from pyspark.sql.window import Window

insights = {}

# Totals
insights["total_apps"] = df.count()
insights["total_installs"] = (
    df.agg(F.sum(F.col("installs_num").cast("double")).alias("s"))
      .first()["s"]
)

# Free vs Paid share
tdc = type_dist_count.withColumn("app_count_d", F.col("app_count").cast("double"))
tot_apps = tdc.agg(F.sum("app_count_d").alias("t")).first()["t"] or 1.0

insights["free_share_apps"] = float(
    tdc.filter(F.col("type_norm") == "Free")
       .select((F.col("app_count_d") / tot_apps))
       .first()[0] or 0.0
)

tdi = type_dist_installs
tot_inst = tdi.agg(F.sum("total_installs").alias("t")).first()["t"] or 1.0

insights["free_share_installs"] = float(
    tdi.filter(F.col("type_norm") == "Free")
       .select((F.col("total_installs") / tot_inst))
       .first()[0] or 0.0
)

# Top category by installs
top_cat_row = category_installs.select("category_norm", "total_installs").first()
if top_cat_row:
    insights["top_category"] = top_cat_row["category_norm"] or "Unknown"
    insights["top_category_installs"] = float(top_cat_row["total_installs"])

# Avg rating: Free vs Paid
avg_by_type = (
    df.groupBy("type_norm")
      .agg(F.avg(F.col("rating_num").cast("double")).alias("avg_rating"))
)

insights["avg_rating_free"] = avg_by_type.filter(F.col("type_norm") == "Free").select("avg_rating").first()
insights["avg_rating_free"] = float(insights["avg_rating_free"][0]) if insights["avg_rating_free"] else None

insights["avg_rating_paid"] = avg_by_type.filter(F.col("type_norm") == "Paid").select("avg_rating").first()
insights["avg_rating_paid"] = float(insights["avg_rating_paid"][0]) if insights["avg_rating_paid"] else None

# Best paid app by rating (with MIN_REVIEWS)
best_paid = (
    top_paid_by_rating
        .select("App", "rating_num", "reviews_num", "price_num", "category_norm")
        .first()
)

if best_paid:
    insights["best_paid_app"] = dict(
        app = best_paid["App"],
        rating = float(best_paid["rating_num"]) if best_paid["rating_num"] is not None else None,
        reviews = int(best_paid["reviews_num"]) if best_paid["reviews_num"] is not None else None,
        price = float(best_paid["price_num"]) if best_paid["price_num"] is not None else None,
        category = best_paid["category_norm"] or "Unknown"
    )

# Simple correlation (paid only): price vs rating (if enough points)
paid_clean = paid_df.select("price_num", "rating_num").dropna()
corr = None
try:
    if paid_clean.count() >= 3:
        corr = paid_clean.stat.corr("price_num", "rating_num")
except Exception:
    corr = None

insights["corr_price_rating_paid"] = corr

insights


{'total_apps': 10840,
 'total_installs': 167627433495.0,
 'free_share_apps': 0.9259225092250922,
 'free_share_installs': 0.999564772379563,
 'top_category': 'GAME',
 'top_category_installs': 35086024415.0,
 'avg_rating_free': 4.186176436847552,
 'avg_rating_paid': 4.266615146831529,
 'best_paid_app': {'app': 'The Room: Old Sins',
  'rating': 4.9,
  'reviews': 21119,
  'price': 4.99,
  'category': 'GAME'},
 'corr_price_rating_paid': -0.11295103131908668}

In [ ]:
!pip install reportlab


  Using cached reportlab-4.4.4-py3-none-any.whl.metadata (1.7 kB)
Using cached reportlab-4.4.4-py3-none-any.whl (2.0 MB)


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

pdf_file = "/content/PlayStore_Report.pdf"

styles = getSampleStyleSheet()
H1, H2, H3, N = styles["Heading1"], styles["Heading2"], styles["Heading3"], styles["Normal"]

def spark_to_table(sdf, max_rows=10):
    _pdf = sdf.limit(max_rows).toPandas()
    _pdf = _pdf.fillna("")   # avoid None issues
    data = [list(map(str, _pdf.columns))] + _pdf.astype(str).values.tolist()
    t = Table(data, repeatRows=1)
    t.setStyle(TableStyle([
        ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
        ("TEXTCOLOR", (0,0), (-1,0), colors.black),
        ("GRID", (0,0), (-1,-1), 0.25, colors.grey),
        ("FONTSIZE", (0,0), (-1,-1), 8),
        ("ALIGN", (1,1), (-1,-1), "LEFT"),
        ("VALIGN", (0,0), (-1,-1), "MIDDLE"),
    ]))
    return t

doc = SimpleDocTemplate(pdf_file, pagesize=A4)
story = []

# Title & Problem Statement
story += [
    Paragraph("Google Play Store Analysis – PySpark on Google Colab", H1),
    Spacer(1, 6),
    Paragraph("<b>Problem Statement</b>", H2),
    Paragraph("""
Analyze the Google Play Store dataset using PySpark in Google Colab and present:
1) Top 10 Apps by Reviews
2) Top 10 Apps by Installs and Free vs Paid distribution
3) Category-wise distribution of installs
4) Top Paid Apps by Installs
5) Top Paid Apps by Rating (with review threshold)
""", N),
    Spacer(1, 10)
]

# Steps
story += [
    Paragraph("Step 1: Load with Explicit Schema", H2),
    Paragraph("We load the Excel as CSV and ingest into Spark with an explicit schema to avoid mixed-type inference errors.", N),
    Spacer(1, 8),
    Paragraph("Step 2: Clean & Normalize", H2),
    Paragraph("We normalize installs, price, reviews, rating, and tidy Type/Category. Ratings constrained to [0,5] when present.", N),
    Spacer(1, 8),
    Paragraph("Step 3: Compute Agenda Queries", H2),
    Paragraph("We build the key aggregations and top-k selections required by the agenda.", N),
    Spacer(1, 10),
]

# Output sections
sections = [
    ("Top 10 Apps by Reviews", top10_reviews, charts["Top 10 Apps by Reviews"]),
    ("Top 10 Apps by Installs", top10_installs, charts["Top 10 Apps by Installs"]),
    ("Free vs Paid (Count)", type_dist_count, charts["Free vs Paid (Count)"]),
    ("Free vs Paid (Installs)", type_dist_installs, charts["Free vs Paid (Installs)"]),
    ("Top Categories by Installs", category_installs, charts["Top Categories by Installs"]),
    ("Top Paid Apps by Installs", top_paid_by_installs, charts["Top Paid Apps by Installs"]),
    ("Top Paid Apps by Rating", top_paid_by_rating, charts["Top Paid Apps by Rating"]),
]

for title, sdf, chart_path in sections:
    story += [
        Paragraph(title, H2),
        spark_to_table(sdf, max_rows=10),
        Spacer(1,6),
        Image(chart_path, width=400, height=260),
        Spacer(1,10)
    ]

# Conclusions & Insights
def fmt_pct(x):
    return f"{x*100:.1f}%" if x is not None else "n/a"

story += [Paragraph("Conclusions & Insights", H2)]
cons_txt = []

cons_txt.append(f"Total apps analyzed: <b>{insights.get('total_apps','n/a')}</b>.")

if insights.get("total_installs") is not None:
    cons_txt.append(f"Total installs (sum): <b>{int(insights['total_installs']):,}</b>.")

cons_txt.append(
    f"Free apps share (by app count): <b>{fmt_pct(insights.get('free_share_apps'))}</b>; "
    f"Free apps share (by installs): <b>{fmt_pct(insights.get('free_share_installs'))}</b>."
)

if insights.get("top_category"):
    cons_txt.append(
        f"Top category by total installs: <b>{insights['top_category']}</b> "
        f"with <b>{int(insights['top_category_installs']):,}</b> installs."
    )

if insights.get("avg_rating_free") is not None or insights.get("avg_rating_paid") is not None:
    if insights.get('avg_rating_free') is not None and insights.get('avg_rating_paid') is not None:
        cons_txt.append(
            f"Average rating — Free: <b>{insights['avg_rating_free']:.2f}</b>, "
            f"Paid: <b>{insights['avg_rating_paid']:.2f}</b>."
        )
    else:
        cons_txt.append(
            f"Average rating — Free: <b>{insights.get('avg_rating_free','n/a')}</b>, "
            f"Paid: <b>{insights.get('avg_rating_paid','n/a')}</b>."
        )

bp = insights.get("best_paid_app")
if bp:
    cons_txt.append(
        f"Best paid app by rating (≥ {MIN_REVIEWS} reviews): <b>{bp['app']}</b> "
        f"(Rating <b>{bp['rating']:.2f}</b>, Reviews <b>{bp['reviews']:,}</b>, "
        f"Price <b>${bp['price']:.2f}</b> if listed, Category <b>{bp['category']}</b>)."
    )

cpr = insights.get("corr_price_rating_paid")
if cpr is not None:
    cons_txt.append(
        f"Correlation (paid apps) between price and rating: <b>{cpr:.2f}</b> "
        "(near 0 implies weak linear relationship)."
    )

for p in cons_txt:
    story += [Paragraph(p, N), Spacer(1,4)]

# Build PDF
doc.build(story)
print("PDF saved:", pdf_file)


PDF saved: /content/PlayStore_Report.pdf


In [ ]:
from google.colab import files
files.download("/content/PlayStore_Report.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>